In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import pandas as pd
pd.options.display.max_colwidth = 1000

path = "C:/Users/Nathan.y.kelmers/Desktop/GA/FINAL PROJECT/data/mbti_combined.csv"
mbti = pd.read_csv(path, header = 0)

import re

In [2]:
def clean_title_1(row):
    
    # remove formatting created by reading a list as a string
    row = row.strip("[").strip("]")
    
    # restore list property by splitting on (', '), (", '), (', "), and (", ") with regex 
    row = re.split("[\"\'], [\"\']", row)
    
    # removes any stray string delimiters from each title, adds clean titles to a new list
    new_row = []
    
    for title in row:
        title = title.strip("'")
        title = title.strip('"')
        
        new_row.append(title)
        
    return new_row

In [3]:
mbti['clean_title_1'] = mbti.youtube_titles.apply(lambda x: clean_title_1(x))

In [4]:
def clean_title_2(row):

    new_row = []
    for title in row:

        # removes all parentheticals and brackets
        title = re.sub("[\(\[\{].*?[\)\]\}]", "", title)

        # removes "MUSIC VIDEO" and "OFFICIAL VIDEO" and variations
        title = re.sub("music video|official video", "", title, flags = re.IGNORECASE)
        
        new_row.append(title)
  
    return new_row

In [5]:
mbti['clean_title_2'] = mbti.clean_title_1.apply(lambda x: clean_title_2(x))

In [6]:
def clean_title_3(row):
    
    new_row = []
    for title in row:
        
        # removes strings representing video attributes
        title = re.sub("HD|HQ", "", title)
        title = re.sub("[A-z]*1080[A-z]*|[A-z]*720[A-z]*", "", title)
        title = re.sub("320[A-z]*", "", title)
        title = re.sub("\.mov|\.wmv|\.m4v|\.mp4|\.flv", "", title)
        title = re.sub("best quality|cd quality|good quality|high quality", "", title, flags = re.IGNORECASE)
        title = re.sub("album version|cd version|english version|long version|official version|original version",
                       "", title, flags = re.IGNORECASE)
        title = re.sub("lyrics|with lyrics|w/.?Lyrics", "", title, flags = re.IGNORECASE)
                
        new_row.append(title)
        
    return new_row

In [7]:
mbti['clean_title_3'] = mbti.clean_title_2.apply(lambda x: clean_title_3(x))

In [37]:
def clean_non_youtube_1(message):
  
    # skips messages with youtube videos
    if "youtube" in message:
        return None

    # Looks for posts that contain songs and not messages by filtering for "by" or "-" (the 
    # most common delineators between the artist and the song)
        # Note: this does miss some poorly formatted songs
    elif "by" in message or "-" in message:

        # skips posts with messages by fuiltering away punctuation
            # Note: this does miss some poorly formatted songs 
        if "." not in message:
            # removes formatting from post
            message = re.sub("<(.*?)>", "", message)

            # removes breaks from post
            message = re.sub("\\r|\\n", "", message)

            # removes Tapatalk app messages from posts
            message = re.sub("Sent from my (.*?) using Tapatalk", "", message)

            return message
        
        else:
            return None

In [38]:
mbti['clean_non_youtube_1'] = mbti.postsoup.apply(lambda x: clean_non_youtube_1(x))

In [73]:
# Need to clean up all of the different "None" values in this column
def clean_nones(row):
    new_row = []
    if type(row) == str:
        if row == ['None']:
            new_row = []
        elif row == 'None':
            new_row = []
        else:
            new_row = row
    elif type(row) == list:
        if row == [None]:
            new_row = []
        elif row == ['None']:
            new_row = []
        elif row == ['']:
            new_row = []
        elif row == None:
            new_row = []
        elif [] in row:
            new_row = []
        else:
            new_row = row
    return new_row

In [74]:
mbti['titles_posts'] = mbti.clean_non_youtube_1.apply(lambda x: clean_nones(x))
mbti['titles_youtube'] = mbti.clean_title_3.apply(lambda x: clean_nones(x))

In [61]:
def clean_nones_again(row):
    new_row = []
    if row == []:
        new_row = None
    if row == ['']:
        new_row = None
    else:
        new_row = row
    return new_row

In [57]:
mbti['titles_posts'] = mbti.titles_posts.apply(lambda x: clean_nones_again(x))
mbti['titles_youtube'] = mbti.titles_youtube.apply(lambda x: clean_nones_again(x))

In [76]:
mbti['titles_posts'] = [[i] if i else [] for i in mbti['titles_posts']]
mbti

,post,username,mbti,postsoup,youtube,youtube_titles,clean_title_1,clean_title_2,clean_title_3,clean_non_youtube_1,clean_non_youtube_2,titles_posts,titles_youtube,titles_all
0,1,crazysah,Unknown Personality,What Song are you listening to now?<br/> <br/>\r\r\r\nI am currently listening to Oasis - Falling Down<br/> <br/>\r\r\r\nCrazysah,NaN,None,[None],[None],[None],What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah,What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah,[What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah],[],NaN
1,2,happy,Unknown,"Austin Powers - Daddy wasn't there<br/> <br/> <iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen61"" src=""//www.youtube.com/embed/GuTevbwE49A?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe>",GuTevbwE49A,"[""Daddy Wasn't There""]",[Daddy Wasn't There],[Daddy Wasn't There],[Daddy Wasn't There],None,[],[],[Daddy Wasn't There],NaN
2,3,snail,INFP - The Idealists,"I'm listening to my Moody Blues box set bonus disc, because the flute part in the live version of Timothy Leary's Dead is amazing.",NaN,None,[None],[None],[None],None,[],[],[],NaN
3,4,trope,INTP - The Thinkers,"<iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen62"" src=""//www.youtube.com/embed/yqnKbdqjh2I?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe>",yqnKbdqjh2I,['LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip (OFFICIAL)'],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip (OFFICIAL)],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ],None,[],[],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ],NaN
4,5,snail,INFP - The Idealists,I'm not listening to music for a change. It is worthy of being mentioned only because it is rare for me to experience silence.,NaN,None,[None],[None],[None],None,[],[],[],NaN
5,6,nephilimazrael,INTJ - The Scientists,"<a href=""http://uk.youtube.com/watch?v=bR-Sl71N-So"" rel=""nofollow"" target=""_blank"">People are People</a> - Depeche Mode",bR-Sl71N-So,[],[],[],[],None,[],[],[],NaN
6,7,lunniey,INFJ - The Protectors,currently listen to jason mraz album...<br/>\r\r\r\ni just love his vocal..,NaN,None,[None],[None],[None],None,[],[],[],NaN
7,8,crazysah,Unknown Personality,Better Than This - Keane,NaN,None,[None],[None],[None],Better Than This - Keane,Better Than This - Keane,[Better Than This - Keane],[],NaN
8,9,sandova,NaN,"<iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen63"" src=""//www.youtube.com/embed/Eg30ps__yEQ?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe> <br/> <br/>\r\r\r\nI love secret romance.. I'm hopeless <img alt="""" border=""0"" class=""inlineimg"" src=""http://personalitycafe.com/images/smilies/1/blushed.gif"" title=""Blushed""/>",Eg30ps__yEQ,[],[],[],[],None,[],[],[],NaN
9,10,sandova,NaN,"And also... <br/> <br/> <iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen64"" src=""//www.youtube.com/embed/-KnwOuWici8?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe>",#NAME?,['Feist - Secret Heart (Live @ The Rehearsal Hall)'],[Feist - Secret Heart (Live @ The Rehearsal Hall)],[Feist - Secret Heart ],[Feist - Secret Heart ],None,[],[],[Feist - Secret Heart ],NaN


In [77]:
mbti["titles_all"] = mbti["titles_posts"] + mbti["titles_youtube"]
mbti

,post,username,mbti,postsoup,youtube,youtube_titles,clean_title_1,clean_title_2,clean_title_3,clean_non_youtube_1,clean_non_youtube_2,titles_posts,titles_youtube,titles_all
0,1,crazysah,Unknown Personality,What Song are you listening to now?<br/> <br/>\r\r\r\nI am currently listening to Oasis - Falling Down<br/> <br/>\r\r\r\nCrazysah,NaN,None,[None],[None],[None],What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah,What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah,[What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah],[],[What Song are you listening to now? I am currently listening to Oasis - Falling Down Crazysah]
1,2,happy,Unknown,"Austin Powers - Daddy wasn't there<br/> <br/> <iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen61"" src=""//www.youtube.com/embed/GuTevbwE49A?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe>",GuTevbwE49A,"[""Daddy Wasn't There""]",[Daddy Wasn't There],[Daddy Wasn't There],[Daddy Wasn't There],None,[],[],[Daddy Wasn't There],[Daddy Wasn't There]
2,3,snail,INFP - The Idealists,"I'm listening to my Moody Blues box set bonus disc, because the flute part in the live version of Timothy Leary's Dead is amazing.",NaN,None,[None],[None],[None],None,[],[],[],[]
3,4,trope,INTP - The Thinkers,"<iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen62"" src=""//www.youtube.com/embed/yqnKbdqjh2I?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe>",yqnKbdqjh2I,['LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip (OFFICIAL)'],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip (OFFICIAL)],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ],None,[],[],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ],[LOOK FOR THE WOMAN - dan le sac Vs Scroobius Pip ]
4,5,snail,INFP - The Idealists,I'm not listening to music for a change. It is worthy of being mentioned only because it is rare for me to experience silence.,NaN,None,[None],[None],[None],None,[],[],[],[]
5,6,nephilimazrael,INTJ - The Scientists,"<a href=""http://uk.youtube.com/watch?v=bR-Sl71N-So"" rel=""nofollow"" target=""_blank"">People are People</a> - Depeche Mode",bR-Sl71N-So,[],[],[],[],None,[],[],[],[]
6,7,lunniey,INFJ - The Protectors,currently listen to jason mraz album...<br/>\r\r\r\ni just love his vocal..,NaN,None,[None],[None],[None],None,[],[],[],[]
7,8,crazysah,Unknown Personality,Better Than This - Keane,NaN,None,[None],[None],[None],Better Than This - Keane,Better Than This - Keane,[Better Than This - Keane],[],[Better Than This - Keane]
8,9,sandova,NaN,"<iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen63"" src=""//www.youtube.com/embed/Eg30ps__yEQ?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe> <br/> <br/>\r\r\r\nI love secret romance.. I'm hopeless <img alt="""" border=""0"" class=""inlineimg"" src=""http://personalitycafe.com/images/smilies/1/blushed.gif"" title=""Blushed""/>",Eg30ps__yEQ,[],[],[],[],None,[],[],[],[]
9,10,sandova,NaN,"And also... <br/> <br/> <iframe class=""restrain"" frameborder=""0"" height=""390"" id=""yui-gen64"" src=""//www.youtube.com/embed/-KnwOuWici8?wmode=opaque"" title=""YouTube video player"" width=""640""></iframe>",#NAME?,['Feist - Secret Heart (Live @ The Rehearsal Hall)'],[Feist - Secret Heart (Live @ The Rehearsal Hall)],[Feist - Secret Heart ],[Feist - Secret Heart ],None,[],[],[Feist - Secret Heart ],[Feist - Secret Heart ]


In [78]:
mbti.to_csv('C:\Users\Nathan.y.kelmers\Desktop\GA\FINAL PROJECT\data\mbti_titles_dirty.csv')

In [79]:
mbti.drop(['postsoup', 
           'youtube', 
           'youtube_titles', 
           'clean_title_1', 
           'clean_title_2', 
           'clean_title_3', 
           'clean_non_youtube_1', 
           'clean_non_youtube_2', 
           'titles_posts', 
           'titles_youtube'], axis = 1, inplace = True) 

In [81]:
mbti.to_csv('C:\Users\Nathan.y.kelmers\Desktop\GA\FINAL PROJECT\data\mbti_titles_clean.csv')